<a href="https://colab.research.google.com/github/Whoami666/HSE_Lab/blob/main/clean_models/schiz_models_svm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [3]:
!pip install scikit-optimize

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.3/100.3 KB 9.0 MB/s eta 0:00:00


In [4]:
%cd drive
%cd MyDrive
%cd shiz

/content/drive
/content/drive/MyDrive
/content/drive/MyDrive/shiz


In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings

In [20]:
def calculate_scores(model, givenx, giveny, xtest):
  kf = KFold(n_splits=10)
  crossval_f1 = cross_val_score(model, givenx, giveny, scoring='f1_macro', cv = kf, verbose=False)
  mean_score = crossval_f1.mean()
  std_score = crossval_f1.std()
  print('f1_macro', mean_score, '±', std_score)

  crossval_accuracy = cross_val_score(model, givenx, giveny, scoring='accuracy', cv = kf, verbose=False)
  mean_score = crossval_accuracy.mean()
  std_score = crossval_accuracy.std()
  print('accuracy', mean_score, '±', std_score)

  crossval_recall = cross_val_score(model, givenx, giveny, scoring='recall_macro', cv = kf, verbose=False)
  mean_score = crossval_recall.mean()
  std_score = crossval_recall.std()
  print('recall', mean_score, '±', std_score)

  if giveny.nunique() == 2:
    y_proba = model.predict_proba(xtest)
    print('roc_auc_score', roc_auc_score(y_test, y_proba[:,1]))

In [38]:
def svm_class(xtrain, ytrain, xtest, ytest, xgiven, ygiven):
  warnings.filterwarnings('ignore')
  opt = BayesSearchCV(
      SVC(),
      {
          'C': (1e-6, 1e+6, 'log-uniform'),
          'gamma': (1e-6, 1e+1, 'log-uniform'),
          'degree': (1, 8),  # integer valued parameter
          'kernel': ['linear', 'poly', 'rbf'],  # categorical parameter
      },
      scoring='f1_macro',
      n_iter=32,
      cv=3
  )

  opt.fit(xtrain, ytrain)

  print("val. score: %s" % opt.best_score_)
  print("test score: %s" % opt.score(xtest, ytest), '\n')


  params = opt.best_params_
  svm = SVC(gamma=params['gamma'], degree=params['degree'], C=params['C'], kernel=params['kernel'], probability=True)
  svm.fit(xtrain, ytrain)
  y_pred = svm.predict(xtest)
  print(classification_report(ytest,y_pred))

  calculate_scores(svm, xgiven, ygiven, xtest)


In [26]:
df = pd.read_excel('df_standard_cleaned_21feb.xlsx') 
df1d = pd.read_excel('df_standard_only1dfeatures.xlsx')

df = df.set_index('ID')
df1d = df1d.set_index('ID')
df.head(1)

,sex,tds,chroma_cens_avg,chroma_cens_std,chroma_cqt_avg,chroma_cqt_std,chroma_stft_avg,chroma_stft_std,melspectrogram_avg,melspectrogram_std,...,spectral_flatness_std,spectral_rolloff_avg,spectral_rolloff_std,tempogram_avg,tempogram_std,tonnetz_avg,tonnetz_std,zero_crossing_rate_avg,zero_crossing_rate_std,tds_binary
ID,,,,,,,,,,,,,,,,,,,,,
PD-003,1,0,0.685706,0.428088,0.425039,0.296394,0.339246,0.434412,0.327867,0.348849,...,0.31499,0.397736,0.254215,0.747885,0.719312,0.691479,0.329593,0.197304,0.281692,0


Models

In [27]:
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import roc_auc_score

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split

from sklearn.inspection import permutation_importance

from sklearn.linear_model import LogisticRegression
from skopt import BayesSearchCV
from sklearn.svm import SVC

**Non-binary**

All features

In [28]:
X = df.drop(['tds', 'tds_binary'], axis=1)
y = df.tds

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=16)

In [29]:
svm_class(X_train, y_train, X_test, y_test, X, y)

val. score: 0.6448882061785287
test score: 0.37849462365591396 

              precision    recall  f1-score   support

           0       0.91      0.97      0.94        60
           1       0.25      0.17      0.20         6
           2       0.00      0.00      0.00         2

    accuracy                           0.87        68
   macro avg       0.39      0.38      0.38        68
weighted avg       0.82      0.87      0.84        68

f1_macro 0.6331417897614134 ± 0.31367604021387796
accuracy 0.8588235294117647 ± 0.19187654623882458
recall 0.6514093137254902 ± 0.28860365019062817


Only1D

In [30]:
X = df1d.drop(['tds', 'tds_binary'], axis=1)
y = df1d.tds

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=16)

In [31]:
svm_class(X_train, y_train, X_test, y_test, X, y)

val. score: 0.4888888888888889
test score: 0.348802712439076 

              precision    recall  f1-score   support

           0       0.89      0.90      0.89        60
           1       0.14      0.17      0.15         6
           2       0.00      0.00      0.00         2

    accuracy                           0.81        68
   macro avg       0.34      0.36      0.35        68
weighted avg       0.79      0.81      0.80        68

f1_macro 0.5168868021296141 ± 0.2600700804744179
accuracy 0.7992647058823529 ± 0.18975859960636163
recall 0.5192577030812325 ± 0.2456791923400649


**Binary**

All features

In [39]:
X = df.drop(['tds', 'tds_binary'], axis=1)
y = df.tds_binary

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=16)

In [40]:
svm_class(X_train, y_train, X_test, y_test, X, y)

val. score: 0.7436415365611647
test score: 0.7166666666666667 

              precision    recall  f1-score   support

           0       0.93      0.93      0.93        60
           1       0.50      0.50      0.50         8

    accuracy                           0.88        68
   macro avg       0.72      0.72      0.72        68
weighted avg       0.88      0.88      0.88        68

f1_macro 0.6840274309024309 ± 0.2585041003066152
accuracy 0.8705882352941178 ± 0.16803361008336118
recall 0.6949264705882353 ± 0.25144253071082623
roc_auc_score 0.75


Only 1d-features

In [41]:
X = df1d.drop(['tds', 'tds_binary'], axis=1)
y = df1d.tds_binary

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=16)

In [42]:
svm_class(X_train, y_train, X_test, y_test, X, y)

val. score: 0.610727417951647
test score: 0.7611241217798596 

              precision    recall  f1-score   support

           0       0.94      0.97      0.95        60
           1       0.67      0.50      0.57         8

    accuracy                           0.91        68
   macro avg       0.80      0.73      0.76        68
weighted avg       0.90      0.91      0.91        68

f1_macro 0.7754880536130536 ± 0.2803149456371135
accuracy 0.8882352941176471 ± 0.1905192322435488
recall 0.8040178571428571 ± 0.24120738560396104
roc_auc_score 0.8583333333333334
